In [ ]:
import os 
import time
import tensorflow as tf
tf.enable_eager_execution()

## Import MNIST dataset

In [ ]:
batch_size = 32
tf.set_random_seed(42)

# load mnist images and store them in two seperate tensorflow dataset (train and test)
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = tf.cast(y_train, dtype=tf.int32), tf.cast(y_test, dtype=tf.int32)
dset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(100, reshuffle_each_iteration=True).repeat().batch(batch_size)
dset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(100, reshuffle_each_iteration=True).repeat().batch(1000)

## Model and training procedure

In [ ]:
class Model(tf.keras.Model):
    '''We use a simple model with only one hidden fully connected layer.
    More complex architectures can achieve higher accuracies, see http://yann.lecun.com/exdb/mnist/.'''
    def __init__(self, units=128):
        super(Model, self).__init__()
        self.units = units
        self.W1 = tf.layers.Dense(units, activation=tf.nn.relu)
        self.W2 = tf.layers.Dense(10)
    
    def call(self, _input):
        '''One forward pass of the model with a batch of MNIST images as the input x (shape: [32, 28x28x]) and 
        logits for each class label as the output (shape: [32, 10]).'''
        x = tf.layers.flatten(_input)
        hidden1 = self.W1(x)
        logits = self.W2(hidden1)
        return logits

def loss(logits, labels):
    '''Cross entropy loss between the predicted (soft) assignments and the true target labels.'''
    return tf.reduce_mean(
      tf.nn.sparse_softmax_cross_entropy_with_logits(
          logits=logits, labels=labels))


def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1, output_type=tf.int64)
    labels = tf.cast(labels, tf.int64)
    batch_size = int(logits.shape[0])
    return tf.reduce_sum(
      tf.cast(tf.equal(predictions, labels), dtype=tf.float32)) / batch_size


def train(model, optimizer, dataset, dataset_test, log_interval=100, stop=500):
    """Trains model on `dataset` using `optimizer`."""
    start = time.time()
    for (batch, (images, labels)) in enumerate(dataset):
        
        # 'tape' records operations for automatic differentiation
        with tf.GradientTape() as tape:
            logits = model(images)
            loss_value = loss(logits, labels)
        
        # compute the gradients and use them to optimize the model variables
        grads = tape.gradient(loss_value, model.variables)
        optimizer.apply_gradients(zip(grads, model.variables))
        
        if log_interval and batch % log_interval == 0:
            rate = log_interval / (time.time() - start) if batch > 0 else 1./(time.time() - start)
            print('Step #%d\tLoss: %.6f (%d steps/sec)' % (batch, loss_value, rate))
            start = time.time()
        if batch % 100 == 0:
            test(model, dataset_test)
        if batch >= stop: break
        
def test(model, dataset):
    """Perform an evaluation of `model` on the examples from `dataset`."""
    avg_loss = 0
    avg_accuracy = 0
    for (images, labels) in dataset:
        logits = model(images)
        avg_loss = loss(logits, labels)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, axis=1, output_type=tf.int32),labels), tf.float32))
        break
    print('Test set: Average loss: {0:.3f}, Accuracy: {1:.1f}%\n'.format(avg_loss.numpy(), 100 * accuracy.numpy()))

## Construct the optimizer
In this exercise you will implement three of the most used optimizer in deep learning from scratch: SGD, Adagrad, and Adam. This blog post (http://ruder.io/optimizing-gradient-descent/) gives a great overview over the different methods.

### SGD
$$ 
    \theta^+ = \theta - \eta L(x_{i:i+n}; \theta)
$$

In [ ]:
class SGD:
    def __init__(self, learning_rate=0.01):
        # TODO
        
    def apply_gradients(self, grads_vars):
        # TODO

### Adagrad 
$$
    g_{t,i} = \nabla_\theta L(x_t; \theta_{t,i})\\
    G_{t,i} = \sum_{\tau=0}^t g_{\tau, i}\\
    \theta^+ = \theta - \frac{\eta}{\sqrt{G_{t,i} + \epsilon}} g_{t,i}
$$
initial_accumulator_value $:= \epsilon$

In [ ]:
class Adagrad:
    def __init__(self, learning_rate=0.001, initial_accumulator_value=0.1):
        # TODO
        
    def apply_gradients(self, grads_vars):
        # TODO

### Adam
https://arxiv.org/pdf/1412.6980.pdf see improved algorithm at the end of section 2 (Different to the version explained in the referenced blogpost).
$$
    m_t = \beta_1 m_{t-1} + (1-\beta_1) \nabla_\theta L(x_t; \theta_t)\\
    v_t = \beta_2 v_{t-1} + (1-\beta_2) \left(\nabla_\theta L(x_t; \theta_t)\right) ^2\\
    \eta_t = \eta * \frac{\sqrt{1-\beta_2^t}}{1-\beta_1^t}
$$
$$
    \theta_{t+1} = \theta_t - \frac{\eta_t}{\sqrt{v_t} + \epsilon}m_t
$$

In [ ]:
class Adam:
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08):
        # TODO
        
    def apply_gradients(self, grads_vars):
        # TODO

# Train the model

In [ ]:
# Set a fixed random seed, so you can compare your implementation against the provided tensorflow optimizer.
tf.set_random_seed(42)

model = Model()

''' Use your own implementation of the optimizer '''
# optimizer = SGD(learning_rate=0.01)
# optimizer = Adagrad(learning_rate=0.1)
# optimizer = Adam(learning_rate=0.01)

''' Or use a pre-built one from tensorflow. Compare your solution against these given optimizers. '''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
# optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
# optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

train(model, optimizer, dset_train, dset_test, stop=500)